In [1]:
import pandas as pd
import numpy as np
import xlwings as xw
import glob
import os

In [50]:
xl = r'C:\Users\emilh\Desktop\Data_split\Full data\AEPrec_sABRe_data_TPsABR21_BILATERAL_2021-Aug-18_13.09.55.xlsx'

try:
    excel_app=xw.App(visible=False)
    wb=excel_app.books.open(xl)
    for sheet in wb.sheets:
        sheet.api.Copy()
        wb_new=xw.books.active
        wb_new.save(f'{xl+sheet.name}.xlsx')
        wb_new.close()
finally:
    excel_app.quit()

In [ ]:
path=r'C:\Users\emilh\Desktop\Data_split'
allfiles=glob.glob(path+"/*.xlsx")
dataframes = []

for files in allfiles:
    df=pd.read_excel(files)
    df.drop(['Time (msec)', 'cond_wa (nV)','rare_wa (nV)', 'contra_wa (nV)'], inplace=True, axis=1)
    #df.insert(0, "Signal", "1")
    for col in df.columns:
        new_df = pd.DataFrame(df[col])
        dataframes.append(new_df)
        new_df.to_excel('data.xlsx')

In [24]:
cwd=os.chdir(r'C:\\Users\\emilh\Desktop\\New_Data_split')
files=os.listdir(cwd)

df=pd.DataFrame()
for file in files:
    if file.endswith('.xlsx'):
        df=df.append(pd.read_excel(file), ignore_index=True)

df.head()
df.to_excel('data.xlsx')

C:\Users\emilh\AppData\Local\Temp\ipykernel_33000\1795859093.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.read_excel(file), ignore_index=True)
C:\Users\emilh\AppData\Local\Temp\ipykernel_33000\1795859093.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.read_excel(file), ignore_index=True)
C:\Users\emilh\AppData\Local\Temp\ipykernel_33000\1795859093.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.read_excel(file), ignore_index=True)
C:\Users\emilh\AppData\Local\Temp\ipykernel_33000\1795859093.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.read_excel(file),

In [9]:
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in 'c:\\Users\\emilh\\Desktop': ['.ipynb_checkpoints', '19-11-2021 Fagrådet velfærdsteknologi Emil Hamberg.pdf', 'Autodesk Fusion 360.lnk', 'Background.docx', 'Background.pdf', 'Content of doctor bag.docx', 'data.xlsx', 'data_engineering.ipynb', 'desktop.ini', 'Faktura_14518.pdf', 'Fan part v1.f3d', 'Feature_extraction.ipynb', 'Full data', 'login info streaming.docx', 'Martin Myrch.docx', 'Martin Myrch.pdf', 'New_Data_split', 'RPG', 'Skole']
